In [2]:
import pandas as pd


In [ ]:
!pip install tensorflow==2.9.2
# pip install pandas

In [58]:
# df=pd.read_csv("E:\Data Science\FakeNewsTwitterProject\ProjectTwitterSentAna\SupervisedNN\SupervisedFakeNews\FakeNewsClassifierLSTMdata.csv")
df=pd.read_csv('/content/FakeNewsClassifierLSTMdata.csv')

In [59]:
df.head()


,text,label
0,ollingStone he man who allegedly attack...,1
1,Dem Rep Sánchez: Trump Has Used ’Fear’ of Musl...,0
2,JasonOn heDrums Ableg News One gro...,1
3,Liberals Who Mocked and Paul For Getting Atta...,1
4,Cathie Wood s fund is collapsing Here s what...,1


In [60]:
df.head()
X=df.drop('label',axis=1)
y=df['label']

In [61]:
X.shape

(1500, 1)

In [62]:
X.head()

,text
0,ollingStone he man who allegedly attack...
1,Dem Rep Sánchez: Trump Has Used ’Fear’ of Musl...
2,JasonOn heDrums Ableg News One gro...
3,Liberals Who Mocked and Paul For Getting Atta...
4,Cathie Wood s fund is collapsing Here s what...


In [63]:
y.shape

(1500,)

In [64]:
# pip install tensorflow

In [65]:
import tensorflow as tf


In [66]:
tf.__version__


'2.9.2'

In [67]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [68]:
voc_size=5000


In [69]:
#One Code
messages = X.copy()

In [70]:
messages['text'][1]

'Dem Rep Sánchez: Trump Has Used ’Fear’ of Muslims And Immigrants To Promote Policies That ’Undermine’ Our Values - Breitbart'

In [71]:
import nltk
import re
from nltk.corpus import stopwords

In [72]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer #for doing stemming porter stemmer 
#lib is used stemming means converting a word to its root word like running,ran,runs to run
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['text'][i]) # here sub is substitute all word apart from ^a-zA-Z will be replaced by ' '
  # now messages['text'][i] for loop will be executed and it will display all the mess in title one by one

  review = review.lower() # mess will be converted into lower cases because Thanks =thanks
  review = review.split() # then splitting the mess

#below line means that if word is in review but not in stopword then only stemming will take place
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review) #message is joined and then appended 
  corpus.append(review)

In [ ]:
corpus

In [ ]:
# now one hot encoding is applied mess is stored in corpus variable now one will give
#  u the index of each word based on voc size assigend earlier as 5000
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

In [ ]:
# now all the sentences are of diff length to make them into fixed length we will do pad sequencing
# padding='pre' means like there is a sentence having onehot_repr as [533,1456,123,456] and [123,456] these both
#sentences are of diff legths so now padding='pre' makes there length 20 by adjusting 0;s in front like 
# [0(16 times),533,1456,123,456]. 
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[5]


In [78]:
embedded_docs.shape

(1500, 20)

In [ ]:
# Creating Model
embedding_vector_features = 10
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length=sent_length))
model.add(LSTM(20))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [80]:
len(embedded_docs),y.shape


(1500, (1500,))

In [81]:
import numpy as np
X_final=np.array(embedded_docs) #independent features [0(16 times),533,1456,123,456] are stored in X by conv it in array.
y_final=np.array(y)

In [82]:
X_final.shape,y_final.shape


((1500, 20), (1500,))

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final , test_size=0.33, random_state=42)

In [ ]:
#Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)


In [85]:
# #Adding DropOut
# from tensorflow.keras.layers import Dropout
# model = Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [86]:
y_pred=model.predict(X_test)


16/16 [==============================] - 0s 3ms/step


In [87]:
y_pred = np.where(model.predict(X_test) > 0.5, 1, 0)


16/16 [==============================] - 0s 3ms/step


In [ ]:
y_pred

In [89]:
from sklearn.metrics import confusion_matrix


In [90]:
confusion_matrix(y_test,y_pred)


array([[147,  12],
       [ 23, 313]])

In [91]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9292929292929293

In [92]:
import pickle

In [93]:
# pip freeze > requirements.txt

In [94]:
model.save("deeplearining.h5")

In [95]:
with open('modelpickle','wb') as f:
  pickle.dump(model,f)

In [100]:
def pre(list):

#     mp = pickle.load(open('E:\\Data Science\\FakeNewsTwitterProject\\ProjectTwitterSentAna\\SupervisedNN\\SupervisedFakeNews\\modelpickle','rb'))
    import keras
    mp = keras.models.load_model("/content/deeplearining.h5")
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    corpus_new2 = []
    for i in range(0, len(list)):

        review = re.sub('[^a-zA-Z]', ' ', list[i])
        review = review.lower()
        review = review.split()

        review = [ps.stem(word)
                  for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus_new2.append(review)

    onehot_repr = [one_hot(words, voc_size)for words in corpus_new2]
    onehot_repr

    sent_length = 20
    embedded_docs_new = pad_sequences(
        onehot_repr, padding='pre', maxlen=sent_length)
    # print(embedded_docs_new)

    unseen_predictions = np.where(mp.predict(embedded_docs_new) > 0.90, 1, 0)

    # unseen_predictions

    output = []
    for i in range(len(unseen_predictions)):
        if unseen_predictions[i] == 1:
            output.append("True")
        else:
            output.append("False")
    return output


In [108]:
# let test with two news records
corpus_new = ['RT @bluevirginia: Haha, @RTDNEWS has video of Youngkin getting booed at the VCU game last night, and someone yelling "I didnt vote for you ', 'Amazon in final talks for the rights to a Warhammer 40,000 series adaptation, with Henry Cavill attached to star and executive ']

output = pre(corpus_new)
output

1/1 [==============================] - 0s 371ms/step


['True', 'False']